# Prepare data for model

In [6]:
from tcn import TCN, tcn_full_summary
import tensorflow as tf
import os
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras import backend as K
import matplotlib.pyplot as plt


def find_max_light(curve):
    id_max = curve[curve["cts"] != 0]['cts'].idxmax()
    max_data = curve.loc[id_max, :]
    return id_max, max_data

if tf.test.gpu_device_name() != '/device:GPU:0':
    print('WARNING: GPU device not found.')
else:
    print('SUCCESS: Found GPU: {}'.format(tf.test.gpu_device_name()))

def plot_model_history(history):
    # summarize history for accuracy
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

data_dir = "./TESS_data/processed_curves/"
files = os.listdir(data_dir)
batch_size = 100
timesteps = 60
n_features = 1

# #format data properly
test_size = int(0.8 * len(files))

def min_max_norm(cts):
    return cts - cts.min() / (cts.max() - cts.min())

def format_data(data, include_uncertainty=False):
    x = []
    y = []

    for csv in data:
        df_x = pd.read_csv(data_dir + csv, index_col="relative_time")
        df_x.index = df_x.index + 1.0
        max_time, max_light = find_max_light(df_x)
        for t in range(df_x.index.size, timesteps+1):
            if not t in df_x.index:
                df_x.loc[t] = [0.0, 0.0]

        if include_uncertainty:
            df_y = np.array([max_time, max_light['e_cts']])
        else:
            df_y = max_time
            df_x = df_x['cts']
            if df_x.shape[0] > 60:
                print(csv)

        x.append(df_x)
        y.append(df_y)

    #reshapes data to be (num_samples, timesteps, num_features), instead of (num_samples, timesteps)
    def add_feature_dim(input):
        print(input.shape)
        return input.reshape(input.shape[0], input.shape[1], n_features)

    x_train = add_feature_dim(np.array(x[:test_size]))
    x_test = add_feature_dim(np.array(x[test_size:]))
    y_train = np.array(y[:test_size])
    y_test = np.array(y[test_size:])

    return x_train, y_train, x_test, y_test

x_train, y_train, x_test, y_test = format_data(files)
# [num_samples, timesteps, num_features)
print(f"input shapes, training:{x_train.shape}")
# [num samples,]
print(f"output shapes, training:{y_train.shape}")

SUCCESS: Found GPU: /device:GPU:0
(2494, 60)
(624, 60)
input shapes, training:(2494, 60, 1)
output shapes, training:(2494,)


In [9]:
print(y_train.max())

58.0


# Simple LSTM
## Training

In [38]:
# model layers
from keras import optimizers
from keras import backend as K

print(np.any(~np.isfinite(x_train)))

model = Sequential()
model.add(tf.keras.layers.Masking(mask_value=0.,
                                  input_shape=(timesteps, n_features)))
model.add(LSTM(50, activation='relu', input_shape=(timesteps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

# fit model
trained = model.fit(x_train, y_train, epochs=200, validation_split=0.2)
model.save("./models/simple")


False
Epoch 1/200
78/78 [==============================] - 62s 770ms/step - loss: 5201422336.0000 - val_loss: 186222256.0000
Epoch 2/200
78/78 [==============================] - 29s 369ms/step - loss: 5398770688.0000 - val_loss: 46938906624.0000
Epoch 3/200
78/78 [==============================] - 28s 353ms/step - loss: 34370633728.0000 - val_loss: 434300032.0000
Epoch 4/200
78/78 [==============================] - 26s 332ms/step - loss: 16739159040.0000 - val_loss: 310691488.0000
Epoch 5/200
78/78 [==============================] - 25s 323ms/step - loss: 11060960256.0000 - val_loss: 51615196.0000
Epoch 6/200
78/78 [==============================] - 22s 278ms/step - loss: 389103040.0000 - val_loss: 41559380.0000
Epoch 7/200
78/78 [==============================] - 22s 288ms/step - loss: 173698465792.0000 - val_loss: 55529668.0000
Epoch 8/200
78/78 [==============================] - 23s 289ms/step - loss: 1604560256.0000 - val_loss: 20462048.0000
Epoch 9/200
78/78 [=====================

KeyError: 'accuracy'

In [42]:
from keras import optimizers

model = Sequential()
model.add(tf.keras.layers.Masking(mask_value=0.,
                                  input_shape=(timesteps, n_features)))
model.add(LSTM(50, activation='relu', input_shape=(timesteps, n_features)))
model.add(Dense(1))
optimizer = optimizers.Adam(clipvalue=1.0)
optimizer.learning_rate = 0.0001
model.compile(optimizer=optimizer, loss='mse')

# fit model
trained = model.fit(x_train, y_train, epochs=200, validation_split=0.2, batch_size=32)
model.save("./models/simple")

Epoch 1/200
78/78 [==============================] - 23s 260ms/step - loss: 3382419652608.0000 - val_loss: 19818905600.0000
Epoch 2/200
78/78 [==============================] - 20s 258ms/step - loss: 97052377088.0000 - val_loss: 196645344.0000
Epoch 3/200
78/78 [==============================] - 20s 259ms/step - loss: 4428836864.0000 - val_loss: 33291460.0000
Epoch 4/200
78/78 [==============================] - 20s 257ms/step - loss: 2220482816.0000 - val_loss: 22289818.0000
Epoch 5/200
78/78 [==============================] - 20s 258ms/step - loss: 18489628672.0000 - val_loss: 72189472.0000
Epoch 6/200
78/78 [==============================] - 20s 257ms/step - loss: 3448571648.0000 - val_loss: 12536550.0000
Epoch 7/200
78/78 [==============================] - 20s 258ms/step - loss: 7987371520.0000 - val_loss: 12430032.0000
Epoch 8/200
78/78 [==============================] - 20s 257ms/step - loss: 6507437568.0000 - val_loss: 381299840.0000
Epoch 9/200
78/78 [===========================

KeyboardInterrupt: 

## Prediction

In [39]:
yhat = model.predict(x_train, verbose=0)
print(yhat)

[[ -10.137939 ]
 [ -21.527594 ]
 [-165.05948  ]
 ...
 [   2.94411  ]
 [   1.7421514]
 [  20.960835 ]]


# Stacked LSTM
## Training

In [13]:
# model layers
stacked_model = Sequential()
stacked_model.add(tf.keras.layers.Masking(mask_value=0.,
                                  input_shape=(timesteps, n_features)))
stacked_model.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(timesteps, n_features)))
stacked_model.add(LSTM(50, activation='relu'))
stacked_model.add(Dense(1))
stacked_model.compile(optimizer='adam', loss='mse')

# fit model
history = stacked_model.fit(x_train, y_train, epochs=200, batch_size=32)


Epoch 1/200
 1/78 [..............................] - ETA: 5:26 - loss: 236655624192.0000

KeyboardInterrupt: 

## Prediction

In [15]:
yhat = stacked_model.predict(x_test, verbose=0)
print(yhat)
stacked_model.history.history.keys()

[[ 1.66204844e+05]
 [ 2.21391816e+04]
 [ 1.02095610e+07]
 [-3.29108359e+04]
 [ 6.62721387e+03]
 [ 7.28425879e+03]
 [ 7.66529724e+02]
 [ 1.44949541e+04]
 [ 2.29155820e+04]
 [-2.04046699e+04]
 [ 1.95377356e+03]
 [ 8.93780391e+04]
 [ 1.80358945e+04]
 [ 4.03771533e+03]
 [ 4.50096016e+04]
 [ 3.32476875e+04]
 [ 9.87614648e+03]
 [ 1.17269617e+03]
 [ 6.76540703e+04]
 [-1.77878510e+02]
 [-1.67209778e+02]
 [ 1.71430359e+01]
 [-7.02323792e+02]
 [ 6.73518828e+04]
 [-9.66421777e+03]
 [ 4.09060195e+04]
 [-9.82723389e+01]
 [ 1.76611328e+02]
 [ 8.16583047e+04]
 [ 2.07215519e+01]
 [ 6.52561914e+03]
 [ 3.09978442e+03]
 [ 1.69183044e+02]
 [ 1.91399216e+02]
 [ 3.14218625e+05]
 [ 4.94676285e+01]
 [ 1.05616766e+05]
 [ 4.50785531e+05]
 [ 4.15685107e+03]
 [ 2.11975952e+03]
 [ 1.84379922e+04]
 [ 1.26441589e+02]
 [ 7.14608398e+03]
 [ 2.62070984e+02]
 [ 5.81176611e+03]
 [-3.82126801e+02]
 [ 7.98217224e+02]
 [ 3.62462695e+04]
 [ 2.08336688e+05]
 [-2.45978928e+02]
 [-7.01234283e+01]
 [-1.47476501e+03]
 [ 1.0947316

dict_keys([])